In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [2]:
file = open('data.txt','r',encoding = "utf8")

In [3]:
lines = []
for i in file:
  lines.append(i)

In [4]:
data = ""
for i in lines:
  data = ' '.join(lines)

In [5]:
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

In [6]:
data = data.split()
data = ' '.join(data)

In [7]:
data[:300]

'THE ADVENTURES OF SHERLOCK HOLMES Arthur Conan Doyle Table of contents A Scandal in Bohemia The Red-Headed League A Case of Identity The Boscombe Valley Mystery The Five Orange Pips The Man with the Twisted Lip The Adventure of the Blue Carbuncle The Adventure of the Speckled Band The Adventure of t'

In [8]:
len(data)

559685

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])


In [10]:
sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 1561, 5, 129, 34, 647, 4498, 4499, 226, 5, 1562, 6, 827, 7, 871]

In [11]:
len(sequence_data)

105879

In [12]:
vocab_size = len(tokenizer.word_index)+1
vocab_size

8200

In [13]:
sequence = []

for i in range(3,len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequence.append(words)

print('The length of sequence: ',len(sequence))
sequences = np.array(sequence)

The length of sequence:  105876


In [14]:
sequences[:1]

array([[   1, 1561,    5,  129]])

In [15]:
sequences.shape

(105876, 4)

In [16]:
X = []
y = []

for i in sequences:
  X.append(i[0:3])
  y.append(i[3])


In [17]:
type(X)

list

In [18]:
X

[array([   1, 1561,    5]),
 array([1561,    5,  129]),
 array([  5, 129,  34]),
 array([129,  34, 647]),
 array([  34,  647, 4498]),
 array([ 647, 4498, 4499]),
 array([4498, 4499,  226]),
 array([4499,  226,    5]),
 array([ 226,    5, 1562]),
 array([   5, 1562,    6]),
 array([1562,    6,  827]),
 array([  6, 827,   7]),
 array([827,   7, 871]),
 array([  7, 871,   1]),
 array([871,   1, 234]),
 array([  1, 234, 462]),
 array([234, 462, 648]),
 array([462, 648,   6]),
 array([648,   6, 110]),
 array([  6, 110,   5]),
 array([ 110,    5, 2072]),
 array([   5, 2072,    1]),
 array([2072,    1,  678]),
 array([   1,  678, 1360]),
 array([ 678, 1360,  499]),
 array([1360,  499,    1]),
 array([499,   1, 294]),
 array([  1, 294, 949]),
 array([294, 949, 872]),
 array([949, 872,   1]),
 array([872,   1,  56]),
 array([ 1, 56, 18]),
 array([56, 18,  1]),
 array([  18,    1, 1014]),
 array([   1, 1014,  873]),
 array([1014,  873,    1]),
 array([873,   1, 577]),
 array([  1, 577,   5]),
 a

In [19]:
X = np.array(X)
y = np.array(y)

In [20]:
type(X)

numpy.ndarray

In [21]:
X.shape

(105876, 3)

In [22]:
y

array([ 129,   34,  647, ..., 8199, 3187, 3186])

In [23]:
y.shape

(105876,)

In [24]:
y = to_categorical(y,num_classes = vocab_size)

In [25]:
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = 3))
model.add(LSTM(1000,return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [28]:
checkpoint = ModelCheckpoint("next_word.keras",monitor='loss',verbose = 1, save_best_only=True)

In [29]:
model.compile(Adam(learning_rate=0.001),loss='categorical_crossentropy')

In [ ]:
model.fit(X,y,epochs = 70,batch_size=64,callbacks=[checkpoint])

Epoch 1/70
1654/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 6.6453
Epoch 1: loss improved from inf to 6.33241, saving model to next_word.keras
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 51s 24ms/step - loss: 6.6449
Epoch 2/70
1654/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.7936
Epoch 2: loss improved from 6.33241 to 5.74993, saving model to next_word.keras
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 84s 28ms/step - loss: 5.7935
Epoch 3/70
1653/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.4400
Epoch 3: loss improved from 5.74993 to 5.43672, saving model to next_word.keras
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 82s 28ms/step - loss: 5.4400
Epoch 4/70
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 5.1876
Epoch 4: loss improved from 5.43672 to 5.18230, saving model to next_word.keras
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 82s 28ms/step - loss: 5.1876
Epoch 5/70
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 4.9474
Epoch 5: loss improved from 5.18230 to 4.95410, saving model to next_word.keras
1655/1655 ━━━━